<a href="https://colab.research.google.com/github/Ronnypetson/titanic/blob/master/produto_interno_maromba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.optim import Adam

import pandas as pd
from sklearn.metrics import accuracy_score
import matplotlib.pylab as plt
import time
from IPython import display
from IPython.core.debugger import Pdb

def breakpoint():
    Pdb().set_trace()

device = "cuda:0" if torch.cuda.is_available() else "cpu"

%matplotlib inline

### Implement example of "Produto Interno Maromba": fitting linear transform using maromba product

#### ----

In [ ]:
class MTensor:
  def __init__(
      self,
      values: torch.Tensor,
      indices: torch.Tensor,
      indexer: nn.Module,
    ):
    assert values.shape == indices.shape[:-1]
    self.data = values
    self.idx = indices
    self.indexer = indexer

  def _gbmd(self, u, v, idxu, idxv) -> torch.Tensor:
    """
    'General Batch Maromba Dot'
    Shorter implementation for the 'batch maromba dot' operation.
    u: M x d_u
    v: N x d_v
    idxu: M x d_u x d_idx
    idxv: N x d_v x d_idx
    """
    m, d_u = u.shape
    n, d_v = v.shape
    d_idx = idxu.shape[-1]
    assert (m, d_u, d_idx) == idxu.shape
    assert (n, d_v, d_idx) == idxv.shape
    # uidxu: M x d_idx
    # vidxv: N x d_idx
    uidxu = torch.bmm(u.reshape(m, 1, d_u), idxu).squeeze(1)
    vidxv = torch.bmm(v.reshape(n, 1, d_v), idxv).squeeze(1)
    dot = uidxu @ vidxv.T
    return dot

  def _genidx(self, idxu, idxv):
    """
    idxu: M x d_u x d_idx
    idxv: N x d_v x d_idx
    indexer: (shape x d_idx) -> (shape x d_idx)
    """
    m, d_u, d_idx = idxu.shape
    n, d_v, _ = idxv.shape
    assert d_idx == idxv.shape[-1]
    # idxu_new: M x d_idx
    # idxv_new: N x d_idx
    idxu_new = (
        self.indexer(idxu.reshape(-1, d_idx))
        .reshape(m, d_u, d_idx).mean(dim=1)
    )
    idxv_new = (
        self.indexer(idxv.reshape(-1, d_idx))
        .reshape(n, d_v, d_idx).mean(dim=1)
    )
    idxu_new = idxu_new.unsqueeze(1).repeat(1, n, 1)
    idxv_new = idxv_new.unsqueeze(0).repeat(m, 1, 1)
    idx_new = idxu_new + idxv_new
    # idx_new = nn.functional.gumbel_softmax(idx_new, hard=True, dim=-1)
    return idx_new

  def __matmul__(self, b):
    apre = self.data.shape[:-1]
    bpre = b.data.shape[:-1]
    d_idx = self.idx.shape[-1]
    assert d_idx == b.idx.shape[-1]
    aidx = self.idx.reshape(*((-1,) + self.idx.shape[-2:]))
    bidx = b.idx.reshape(*((-1,) + b.idx.shape[-2:]))
    mdot = self._gbmd(
        self.data.reshape(-1, self.data.shape[-1]),
        b.data.reshape(-1, b.data.shape[-1]),
        aidx,
        bidx
    )
    mdot = mdot.reshape(apre + bpre)
    midx = self._genidx(aidx, bidx)
    midx = midx.reshape(apre + bpre + (d_idx,))
    mans = MTensor(mdot, midx, self.indexer)
    return mans

#### -------

In [ ]:
def dotlike(
    a: torch.Tensor,
    b: torch.Tensor,
    op=lambda x, y: x * y,
  ) -> torch.Tensor:
  """
  a: ... x d
  b: d x ...
  """
  d = a.shape[-1]
  N = a.shape[:-1]
  M = b.shape[1:]
  assert d == b.shape[0]
  a = a.reshape(-1, d)
  # b: ... x d
  b = b.reshape(d, -1).T
  N_ = a.shape[0]
  M_ = b.shape[0]
  a = a.unsqueeze(1).repeat(1, M_, 1)
  b = b.unsqueeze(0).repeat(N_, 1, 1)
  dot = op(a, b).sum(dim=-1)
  dot = dot.reshape(N + M)
  return dot

def y(x, W):
  """
  x: N x d_in
  W: d_out x d_in
  """
  return x @ W.T

def combine_indices(index_w, index_x, indexer):
  """
  index_w: d_out x d_in x d_index
  index_x: N x d_in x d_index
  index_new[i, j] = f(index_w.sum(dim=1)[i] + index_x.sum(dim=1)[j]; theta)
  """
  d_out, d_in, d_index = index_w.shape
  n, d_in_, d_index_ = index_x.shape
  assert d_in == d_in_
  assert d_index == d_index_
  idxw_sum = index_w.sum(dim=1)
  # idxw_sum: d_out x N x d_index
  idxw_sum = idxw_sum.unsqueeze(1).repeat(1, n, 1)
  idxx_sum = index_x.sum(dim=1)
  # idxx_sum: d_out x N x d_index
  idxx_sum = idxx_sum.unsqueeze(0).repeat(d_out, 1, 1)
  index_new = indexer(idxw_sum + idxx_sum)
  # index_new: N x d_out x d_index
  index_new = index_new.permute(1, 0, 2)
  index_new = nn.functional.gumbel_softmax(index_new, hard=False, dim=-1)
  # index_new = nn.functional.softmax(index_new, dim=-1)
  return index_new

def genidx(idxu, idxv, indexer, debug=False):
  """
  idxu: M x d_u x d_idx
  idxv: N x d_v x d_idx
  indexer: (shape x d_idx) -> (shape x d_idx)
  """
  m, d_u, d_idx = idxu.shape
  n, d_v, _ = idxv.shape
  assert d_idx == idxv.shape[-1]
  # idxu_new: M x d_idx
  # idxv_new: N x d_idx
  if debug:
    Pdb().set_trace()
  idxu_new = indexer(idxu.reshape(-1, d_idx)).reshape(m, d_u, d_idx).mean(dim=1)
  idxv_new = indexer(idxv.reshape(-1, d_idx)).reshape(n, d_v, d_idx).mean(dim=1)
  idxu_new = idxu_new.unsqueeze(1).repeat(1, n, 1)
  idxv_new = idxv_new.unsqueeze(0).repeat(m, 1, 1)
  idx_new = idxu_new + idxv_new
  idx_new = nn.functional.gumbel_softmax(idx_new, hard=True, dim=-1)
  return idx_new

def gbmd(u, v, idxu, idxv, indexer):
  """
  'General Batch Maromba Dot'
  Shorter implementation for the 'batch maromba dot' operation.
  u: M x d_u
  v: N x d_v
  idxu: M x d_u x d_idx
  idxv: N x d_v x d_idx
  indexer: (shape x d_idx) -> (shape x d_idx)
  """
  m, d_u = u.shape
  n, d_v = v.shape
  d_idx = idxu.shape[-1]
  assert (m, d_u, d_idx) == idxu.shape
  assert (n, d_v, d_idx) == idxv.shape
  # uidxu: M x d_idx
  # vidxv: N x d_idx
  uidxu = torch.bmm(u.reshape(m, 1, d_u), idxu).squeeze(1)
  vidxv = torch.bmm(v.reshape(n, 1, d_v), idxv).squeeze(1)
  dot = uidxu @ vidxv.T
  ### Strong reason for encapsulation:
  idx_new = genidx(idxu, idxv, indexer, debug=False)
  # idx_new = combine_indices(idxu, idxv, indexer).permute(1, 0, 2)
  return dot, idx_new

def batch_maromba_dot(W, x, index_w, index_x, indexer):
  """
  W: d_out x d_in
  x: N x d_in
  index_w: d_out x d_in x d_index
  index_x: N x d_in x d_index
  """
  d_out, d_in = W.shape
  n, d_in_ = x.shape
  d_index = index_w.shape[-1]
  assert d_in == d_in_, "W.shape[1] and x.shape[1] must be equal."
  assert index_w.shape[1:] == index_x.shape[1:]
  # comb: N x d_out x d_in x d_in
  W = W.reshape(-1, 1)
  x = x.T.reshape(1, -1)
  comb = (W @ x).reshape(d_out, d_in, d_in, n)
  comb = comb.permute(3, 0, 1, 2)
  # comb: (N * d_out) x (d_in(W) * d_in(x)) x 1
  comb = comb.reshape(n * d_out, d_in * d_in, 1)
  index_x = index_x.permute(2, 1, 0)
  # index_x: d_index x (d_in * N)
  index_x = index_x.reshape(d_index, d_in * n)
  # index_w: (d_out * d_in) x d_index
  index_w = index_w.reshape(d_out * d_in, d_index)
  sim = (index_w @ index_x).reshape(d_out, d_in, d_in, n)
  sim = sim.permute(3, 0, 1, 2)
  sim = sim.reshape(n * d_out, 1, d_in * d_in)
  # Overview of shapes:
  # sim:  (N * d_out) x 1 x (d_in(W) * d_in(x))
  # comb: (N * d_out) x (d_in(W) * d_in(x)) x 1
  # Matrix product of the last two dimensions seems correct.
  dot = torch.bmm(sim, comb)[:, 0, 0]
  dot = dot.reshape(n, d_out)
  index_w = index_w.reshape(d_out, d_in, d_index)
  index_x = index_x.reshape(d_index, d_in, n).permute(2, 1, 0)
  index_new = combine_indices(index_w, index_x, indexer)
  return dot, index_new

def batch_maromba_dot_(u, v, index_u, index_v):
  """
  u: N x dim_u
  v: N x dim_v
  index_u: N x dim_u x index_dim
  index_v: N x dim_v x index_dim
  """
  n, dim_u = u.shape
  m, dim_v = v.shape
  assert n == m, "u.shape[0] and v.shape[0] must be equal."
  u = u.reshape(n, dim_u, 1)
  v = v.reshape(n, 1, dim_v)
  # comb: N x dim_u x dim_v
  comb = torch.bmm(u, v)
  comb = comb.reshape(n, -1, 1)
  index_v = index_v.permute(0, 2, 1)
  # sim: N x dim_u x dim_v
  sim = torch.bmm(index_u, index_v)
  sim = sim.reshape(n, 1, -1)
  # dot: N
  dot = torch.bmm(sim, comb)[:, 0, 0]
  return dot

def maromba_loss(y_true, y_pred, true_index, pred_index, debug=False):
  """
  y_true: N x d_out
  y_pred: N x d_out
  true_index: N x d_out x d_index
  pred_index: N x d_out x d_index
  """
  assert y_true.shape == y_pred.shape
  assert true_index.shape == pred_index.shape
  # dot_true = torch.bmm(y_true.unsqueeze(1), y_true.unsqueeze(-1))[:, 0, 0]
  # dot_pred = torch.bmm(y_pred.unsqueeze(1), y_pred.unsqueeze(-1))[:, 0, 0]
  # mdot_true_pred = batch_maromba_dot_(y_true, y_pred, true_index, pred_index)
  index_match = (pred_index.mean(dim=0) @ true_index.mean(dim=0).T)
  # match_loss_lr = (y_pred - (index_match @ y_true.T).T).abs().mean()
  # match_loss_rl = (y_true - (index_match.T @ y_pred.T).T).abs().mean()
  
  # match_loss_lr = ((y_pred - (y_true @ index_match.T)).abs()).mean()
  # match_loss_rl = ((y_true - (y_pred @ index_match)).abs()).mean()

  huber = nn.HuberLoss()
  match_loss_lr = huber(y_pred, y_true @ index_match.T)
  match_loss_rl = huber(y_true, y_pred @ index_match)

  # dot_loss = ((dot_true - mdot_true_pred).abs()).mean()
  # mu_loss = (y_true.mean(dim=-1) - y_pred.mean(dim=-1)).abs().mean()
  # index_loss_0 = (1.0 - index_match.sum(dim=0)).abs().mean()
  # index_loss_1 = (1.0 - index_match.sum(dim=-1)).abs().mean()
  if debug:
    Pdb().set_trace() ###
  # loss = (dot_loss + mu_loss + index_loss_0 + index_loss_1)
  loss = match_loss_lr + match_loss_rl
  return loss

#### -------

In [ ]:
in_dim = 10
out_dim = 15
index_dim = in_dim + out_dim # making things easier
__hidden_dim = 5 * index_dim
num_examples = 1000

# Ground-truth parameters
W_true = torch.randn((out_dim, in_dim), requires_grad=False)
W_true = W_true.float().to(device)

# Parameters to be trained
bag_values_W = nn.Parameter(torch.randn((out_dim, in_dim)))
bag_values_W = bag_values_W.float().to(device)
bag_indices_W = nn.Parameter(torch.randn((out_dim, in_dim, index_dim)))
bag_indices_W = nn.Parameter(
    torch.eye(index_dim)[:in_dim].unsqueeze(0).repeat(out_dim, 1, 1)
    + torch.eye(index_dim)[in_dim:].unsqueeze(1).repeat(1, in_dim, 1)
)
# bag_indices_W = bag_indices_W.float().to(device)
###
__W = MTensor(bag_values_W, bag_indices_W, indexer)
###

# Indexer model to be trained
# indexer = nn.Sequential(
#     nn.Linear(index_dim, __hidden_dim),
#     # nn.Dropout(0.5),
#     nn.ReLU(),
#     nn.Linear(__hidden_dim, index_dim),
# ).to(device)
indexer = nn.Identity().to(device)

# Input data
values_x = 1e0 * torch.randn((num_examples, in_dim))
# index_x = torch.randn((1, in_dim, index_dim)).repeat(num_examples, 1, 1)
index_x = torch.eye(index_dim)[:in_dim]
index_x = index_x.unsqueeze(0).repeat(num_examples, 1, 1)

# Ground-truth target
y_true = y(values_x, W_true)
# y_true_index = torch.randn((1, out_dim, index_dim)).repeat(num_examples, 1, 1)
y_true_index = torch.eye(index_dim)[in_dim:]
# y_true_index = nn.functional.gumbel_softmax(
#     y_true_index[torch.randperm(out_dim)]
#     + y_true_index[torch.randperm(out_dim)],
#     dim=1,
#     hard=False,
# )
y_true_index = y_true_index.unsqueeze(0).repeat(num_examples, 1, 1)

#### -------

In [ ]:
# __W = MTensor(bag_values_W, bag_indices_W, indexer)
# __x = MTensor(values_x, index_x, indexer)
# %timeit -n 100 mdot = __x @ __W
# %timeit -n 100 mdot = __x.data @ __W.data.T
# # print(mdot.data.shape, mdot.idx.shape)

In [ ]:
# opt_vectors = Adam([bag_values_W, bag_indices_W], lr=1e-2)
opt_vectors = Adam([__W.data], lr=1e-2) # , __W.idx
# opt_vectors = Adam([bag_values_W], lr=1e-1)
# opt_indexer = Adam(indexer.parameters(), lr=1e-2)

num_epochs = 30
batch_size = 32
epoch_len = num_examples // batch_size

all_losses = []
for epoch in range(num_epochs):
  epoch_losses = []
  for _ in range(epoch_len):
    batch_idx = np.random.choice(num_examples, batch_size)
    batch_x = values_x[batch_idx].float().to(device)
    batch_x_index = index_x[batch_idx].float().to(device)
    ###
    __batch_x = MTensor(batch_x, batch_x_index, indexer)
    ###
    batch_y_true = y_true[batch_idx].float().to(device)
    batch_y_true_index = y_true_index[batch_idx].float().to(device)
    # gumbel_bag_indices_W = nn.functional.gumbel_softmax(
    #     bag_indices_W, hard=False, dim=-1
    # ) ###
    # y_pred_val, y_pred_index = batch_maromba_dot(
    #     bag_values_W, batch_x, bag_indices_W, batch_x_index, indexer
    # ) ###
    # y_pred_val, y_pred_index = gbmd(
    #     batch_x, bag_values_W, batch_x_index, bag_indices_W, indexer
    # )
    ###
    __y_pred = __batch_x @ __W
    y_pred_val, y_pred_index = __y_pred.data, __y_pred.idx
    ###
    loss = maromba_loss(
        batch_y_true, y_pred_val, batch_y_true_index, y_pred_index
    )
    opt_vectors.zero_grad()
    # opt_indexer.zero_grad()
    loss.backward()
    opt_vectors.step()
    # opt_indexer.step()
    epoch_losses.append(loss.item())
  all_losses.append(np.mean(epoch_losses))
  df_train = pd.DataFrame({
      "train loss": all_losses,
  })
  display.clear_output(wait=True)
  df_train.plot(figsize=(24, 2))
  plt.show()

In [ ]:
print(np.mean(all_losses[-5:]))
# y_pred_index.shape, batch_y_true_index.shape
index_match = (y_pred_index[0] @ batch_y_true_index[0].T)
print(index_match.sum(dim=-1))
print(index_match.sum(dim=0))

In [ ]:
(W_true - bag_values_W).abs().mean()

In [ ]:
idxs = torch.argmax(y_pred_index[0], dim=-1) - in_dim
(batch_y_true[0][idxs] - y_pred_val[0]).abs().mean()

In [ ]:
batch_y_true[0][idxs], y_pred_val[0]